In [1]:
import aocd
from itertools import permutations
data = list([int(d) for d in aocd.get_data(day=9).split(",")])
print(data)

[1102, 34463338, 34463338, 63, 1007, 63, 34463338, 63, 1005, 63, 53, 1101, 3, 0, 1000, 109, 988, 209, 12, 9, 1000, 209, 6, 209, 3, 203, 0, 1008, 1000, 1, 63, 1005, 63, 65, 1008, 1000, 2, 63, 1005, 63, 904, 1008, 1000, 0, 63, 1005, 63, 58, 4, 25, 104, 0, 99, 4, 0, 104, 0, 99, 4, 17, 104, 0, 99, 0, 0, 1101, 27, 0, 1014, 1101, 286, 0, 1023, 1102, 1, 35, 1018, 1102, 20, 1, 1000, 1101, 26, 0, 1010, 1101, 0, 289, 1022, 1102, 1, 30, 1019, 1102, 734, 1, 1025, 1102, 1, 31, 1012, 1101, 25, 0, 1001, 1102, 1, 1, 1021, 1101, 0, 36, 1002, 1101, 0, 527, 1028, 1101, 895, 0, 1026, 1102, 1, 23, 1016, 1101, 21, 0, 1003, 1102, 22, 1, 1011, 1102, 1, 522, 1029, 1102, 1, 892, 1027, 1102, 1, 0, 1020, 1102, 1, 28, 1015, 1102, 38, 1, 1006, 1101, 0, 32, 1008, 1101, 743, 0, 1024, 1101, 0, 37, 1007, 1102, 1, 24, 1013, 1102, 1, 33, 1009, 1102, 39, 1, 1004, 1102, 1, 34, 1005, 1102, 1, 29, 1017, 109, 19, 21102, 40, 1, -3, 1008, 1016, 40, 63, 1005, 63, 203, 4, 187, 1106, 0, 207, 1001, 64, 1, 64, 1002, 64, 2, 64, 109, 

In [7]:
IN_SIZE = {1: 4, 2: 4, 3: 2, 4: 2, 5: 3, 6: 3, 7: 4, 8: 4, 9: 2, 99: 1}
IN_NAME =  {1: "add", 2: "mul", 3: "rd", 4: "prnt", 5: "jnz", 6: "jz", 
            7: "lt", 8: "eq", 9: "bas", 99: "ret"}
EXT_MEM = 1000

class Process():  # wrapper for generator
    def __init__(self, data, inp=[], dbg=False): 
        self.d = data[:]+[0]*EXT_MEM  # copy + extend memory
        self.done = False
        self.pg = self.process_gen(dbg=dbg)
        self.inp = inp
        self.base = 0
        
    def process(self, inp=[]):
        self.inp = inp
        return next(self.pg)
    
    def proc_data(self, inp): 
        self.process(inp)
        return self.d[:-EXT_MEM]
    
    def parse_ins(self, ptr, dbg):
        param = [0, 0, 0]
        ins = self.d[ptr]%100
        m = [self.d[ptr]//10**e%10 for e in range(2,5)]
        for i, m in enumerate(m):
            size = IN_SIZE[ins]-1
            if i < size:
                if m == 0:  param[i] = self.d[ptr+1+i]          # position
                if m == 1:  param[i] = ptr+1+i                  # intermediate
                if m == 2:  param[i] = self.base+i-1            # relative
        if dbg: print(ptr, IN_NAME[ins], param[:size], 
                      self.d[ptr:ptr+4], sep = "\t")# debug print
        return [ins] + param

    def process_gen(self, dbg): 
        out = []; ptr = 0; d = self.d  # initializations
        parse = lambda i: [i%100] + [i//10**e%10 for e in range(2,5)]
        while ptr < len(d):                                     # stop on EOF
            ins, p1, p2, p3 = self.parse_ins(ptr, dbg=dbg)
            # if dbg:print(ptr, d)                              # debug print
            if   ins == 1: d[p3] = d[p1] + d[p2]                # add
            elif ins == 2: d[p3] = d[p1] * d[p2]                # mul
            elif ins == 3:                                      # read
                if not self.inp: yield out; out = []            # wait/flush
                d[p1] = self.inp.pop(0)                         # read
            elif ins == 4: out.append(d[p1])                    # print
            elif ins == 5: ptr = d[p2]-3 if     d[p1] else ptr  # jnz
            elif ins == 6: ptr = d[p2]-3 if not d[p1] else ptr  # jz
            elif ins == 7: d[p3] = int(d[p1] < d[p2])           # lt
            elif ins == 8: d[p3] = int(d[p1] == d[p2])          # eq
            elif ins == 9: self.base += d[p1]                   # base
            elif ins ==99: self.done=True; yield out            # ret
            else: print(f"invalid opcode {ins} @ {ptr}")        # err
            ptr += IN_SIZE[ins] # jmp is compensated with -3    # move ptr
        

def chain_process(prog, inl, out=[0], dbg=False):
    proc = [Process(prog, dbg=dbg) for inp in inl]
    for epoch in range(100000):
        for i, inp in enumerate(inl):
            out = proc[i].process(([inp] if epoch == 0 else [])+out)
            if proc[i].done and i == len(inl)-1:  # if the last is done, get out
                return out[0]


assert Process([1,9,10,3,2,3,11,0,99,30,40,50]).proc_data([]) == [3500,9,10,70,2,3,11,0,99,30,40,50]
assert Process([1002,4,3,4,33]).proc_data([]) == [1002,4,3,4,99]
assert chain_process([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0], [4,3,2,1,0]) == 43210
assert chain_process([3,23,3,24,1002,24,10,24,1002,23,-1,23,
                      101,5,23,23,1,24,23,23,4,23,99,0,0], [0,1,2,3,4]) == 54321
assert chain_process([3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,
                      1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0], [1,0,4,3,2]) == 65210
assert chain_process([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
                      27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5], [9,8,7,6,5]) == 139629729
assert chain_process([3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
                      -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
                      53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10], [9,7,8,5,6]) == 18216
sp = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
assert Process(sp).process() == sp
assert Process([1102,34915192,34915192,7,4,7,99,0]).process() == [1219070632396864]

In [ ]:
def chain_process_loop(prog, ran=range(5), dbg=False):  # search all permutations
    best = {chain_process(prog, inp, dbg=dbg): inp for inp in permutations(ran)}
    best_k = max(best.keys())
    return best_k, list(best[best_k])  # return (best score, best permutation)

assert chain_process_loop([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]) [0] == 43210
assert chain_process_loop([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
                      27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5], range(5,10)) == (139629729, [9,8,7,6,5])
assert chain_process_loop([3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
                      -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
                      53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10], range(5,10)) == (18216, [9,7,8,5,6])
# assert chain_process_loop(data) == (79723, [2,3,1,0,4])                   # lvl7 checks
# assert chain_process_loop(data, range(5,10)) == (70602018 , [6,9,7,8,5])

In [8]:
Process(data+[0]*1000, dbg=True).process([1])
#aocd.submit(day=7, answer=chain_process_loop(data)[0])

0	mul	[1, 2, 63]	[1102, 34463338, 34463338, 63]
4	lt	[63, 6, 63]	[1007, 63, 34463338, 63]
8	jnz	[63, 10]	[1005, 63, 53, 1101]
11	add	[12, 13, 1000]	[1101, 3, 0, 1000]
15	bas	[16]	[109, 988, 209, 12]
17	bas	[987]	[209, 12, 9, 1000]
19	bas	[1000]	[9, 1000, 209, 6]
21	bas	[990]	[209, 6, 209, 3]
23	bas	[990]	[209, 3, 203, 0]
25	rd	[990]	[203, 0, 1008, 1000]
27	eq	[1000, 29, 63]	[1008, 1000, 1, 63]
31	jnz	[63, 33]	[1005, 63, 65, 1008]
34	eq	[1000, 36, 63]	[1008, 1000, 2, 63]
38	jnz	[63, 40]	[1005, 63, 904, 1008]
41	eq	[1000, 43, 63]	[1008, 1000, 0, 63]
45	jnz	[63, 47]	[1005, 63, 58, 4]
48	prnt	[25]	[4, 25, 104, 0]
50	prnt	[51]	[104, 0, 99, 4]
52	ret	[]	[99, 4, 0, 104]


[203, 0]

In [ ]:
# print(list(itertools.permutations(range(5, 10))))
aocd.submit(day=7, answer=chain_process_loop(data, range(5,10), False)[0])